# Construção de um Classificador Binário baseado no algoritmo KNN

## Dependências necessárias

In [ ]:
# In[50]:
from __future__ import absolute_import, division, print_function
import pandas as pd
import nltk  
import numpy as np  
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
import heapq
import re  
import io
import math
import csv
# Helper libraries
import matplotlib.pyplot as plt
import seaborn as sns
import functools
import operator
import PIL
import tqdm
import tqdm.auto
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.preprocessing import StandardScaler  
from IPython.display import display
import os

## Carregando variáveis de ambiente

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# ENV_DATASET_FILENAME = os.environ['INPUT_DATASET_FILENAME']
# ENV_ORDER_DEGREE = os.environ['ORDER_DEGREE']
# ENV_TRAIN_TEST_PROPORTION = os.environ['TRAIN_TEST_PROPORTION']

ENV_DATASET_FILENAME = "locations_to_be_labeled.csv"
ENV_ORDER_DEGREE = 2845
ENV_TRAIN_TEST_PROPORTION = 0.8

nameOfTheFile = ENV_DATASET_FILENAME
print("ENV_DATASET_FILENAME: Nome do arquivo que contém o dataset: " + nameOfTheFile)
print("ORDER_DEGREE: As " + str(ENV_ORDER_DEGREE) + " palavras mais frequentes da bag of words serão consideradas na construção do modelo.")
print("ENV_TRAIN_TEST_PROPORTION: proporção da divisão do dataset em treino e teste: " + str(ENV_TRAIN_TEST_PROPORTION*100) + "%")

## Preparação do conjunto de dados

### Carregando o Dataset

In [ ]:
# Lendo as features
questions = pd.read_csv("../" + nameOfTheFile, header=0, usecols=[0])
listOfQuestions = []
for row in questions.values:
    listOfQuestions.append(list(row)[0])

print("Quantidade de perguntas: " + str(len(listOfQuestions)) + "\n")
print("Tweet:\n")
for pergunta in listOfQuestions[0:5]:
    print("- " + pergunta)
print("...")
print("\n")

# Lendo as labels
answers = pd.read_csv("../" + nameOfTheFile, header=0, usecols=[1])
listOfAnswers = []
for answer in answers.values:
    listOfAnswers.append(list(answer)[0])
print("Respostas:\n")
for label in listOfAnswers[0:5]:
    print("- " + str(label))
print("...")

### Transformando as labels

Essa transformação vai trocar as labels de número para texto, com o objetivo de facilitar a compreensão e leitura de resultados.
0 - generico
1 - doente

In [ ]:
label_palavras = []
for item in listOfAnswers:
    if item==0:
        label_palavras.append("generico")
    else:
        label_palavras.append("doente")
print(label_palavras[0:5])
listOfAnswers = label_palavras

### Remoção de espaçamentos extras, caracteres especiais e pontuações

In [ ]:
for i in range(len(listOfQuestions)):
    listOfQuestions [i] = listOfQuestions [i].lower()
    listOfQuestions [i] = re.sub(r'\W',' ',listOfQuestions [i])
    listOfQuestions [i] = re.sub(r'\s+',' ',listOfQuestions [i])

for question in listOfQuestions[0:5]:
    print("- " + question)
print("...")

### Tokenização dos tweets

In [ ]:
def tokenize_sentences(listOfSentences):
    listOfSentencesInTokens = []
    for i in range(len(listOfSentences)):
        tokens = nltk.word_tokenize(listOfSentences[i])
        listOfSentencesInTokens.append(tokens)
    return listOfSentencesInTokens


listOfTokenizedQuestions = tokenize_sentences(listOfQuestions)
for tokenizedItem in listOfTokenizedQuestions[0:5]:
      print("- " + str(tokenizedItem))
print("...")

### Remoção das stop-words

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords[:10]
print("Tamanho das stop-words: " + str(len(stopwords)))

def remove_stop_words(listOfTokenizedSentences):
    sentencesWithNoStopWords = []
    for i in range(len(listOfTokenizedSentences)):
        sentenceWithNoStopWord = []
        for j in range(len(listOfTokenizedSentences[i])):
            if(listOfTokenizedSentences[i][j] not in stopwords):
                sentenceWithNoStopWord.append(listOfTokenizedSentences[i][j])
        sentencesWithNoStopWords.append(sentenceWithNoStopWord)
    return sentencesWithNoStopWords

listOfQuestionsWithNoStopWords = remove_stop_words(listOfTokenizedQuestions)
print("Tamanho: " + str(len(listOfQuestionsWithNoStopWords)))

for itemWithNoStopWord in listOfQuestionsWithNoStopWords[0:5]:
    print("- " + str(itemWithNoStopWord))
print("...")

### Lematização das tokens

In [ ]:
stemmer = SnowballStemmer("portuguese")
print()

def lematizar_tokens(listOfTokenizedSentences):
    stemmed_sentences = []
    for tokenizedSentence in listOfTokenizedSentences:
        stemmed_sentence = []
        for token in tokenizedSentence:
            stemmed_sentence.append(stemmer.stem(token))
        stemmed_sentences.append(stemmed_sentence)
    return stemmed_sentences

def remove_redundancies(stemmed_sentences, labels):
    filtered_list = []
    filtered_labels = []
    for index in range(len(stemmed_sentences)):
        isRedundant = False
        for filtered_sentence in filtered_list:
            if(filtered_sentence == stemmed_sentences[index]):
                isRedundant = True
        if(not isRedundant):
            filtered_list.append(stemmed_sentences[index])
            filtered_labels.append(labels[index])
    return {"features": filtered_list, "labels": filtered_labels}

sentencas_lematizadas = lematizar_tokens(listOfQuestionsWithNoStopWords)
filtered_dataset = remove_redundancies(sentencas_lematizadas, listOfAnswers)

sentencas_lematizadas_e_filtradas = filtered_dataset["features"]
labels_lematizadas_e_filtradas = filtered_dataset["labels"]

# print("----------------------------------------------")
# print("Sentenças lematizadas: ")
# print(sentencas_lematizadas)
# print("Tamanho: " + str(len(sentencas_lematizadas)))
# print("\n")

print("Sentenças lematizadas e filtradas: ")
print("Tamanho: " + str(len(sentencas_lematizadas_e_filtradas)))
for stemmedItem in sentencas_lematizadas_e_filtradas[0:5]:
    print("- " + str(stemmedItem))
print("...")
print("\n")

print("Labels lematizadas e filtradas: ")
print("Tamanho: " + str(len(labels_lematizadas_e_filtradas)))
for stemmedLabel in labels_lematizadas_e_filtradas[0:5]:
    print(stemmedLabel)
print("...")
print("\n")

### Construção da bag-of-words

In [ ]:
def create_bag_of_words(listOfTokenizedSentences):
    wordfreq = {}
    for i in range(len(listOfTokenizedSentences)):  #Para cada sentença tokenizada
        for token in listOfTokenizedSentences[i]:       # Para cada token em uma sentença tokenizada
            if token not in wordfreq.keys():
                wordfreq[token] = 1
            else:
                wordfreq[token] += 1
    return wordfreq

wordfreq = create_bag_of_words(sentencas_lematizadas_e_filtradas)
# print(wordfreq)
bag_of_words_size = len(wordfreq.keys())
print("Tamanho da bag of words: " + str(bag_of_words_size))
